## 文本分类步骤
- 划分数据集
- 对标题和正文分词和去停用词
- 计算tf-idf等特征
- 构建分类器

### 从ES取出带标签的数据，分词，并dump到本地

In [13]:
import sys
sys.path.append('..')
import esProxy
from analyzer import Analyzer
import pickle, random

# 从ES导出带标签的新闻数据
sougouNews = esProxy.getDataFromEs()
print('成功导出新闻数据：size=%d' % (len(sougouNews)))

def featurelize(sougouNews, fields=['doc_title'], analyzer=Analyzer()):
    """
    返回标签和分词后的特征
    """
    tokens = []
    for doc in sougouNews:
        dic = {}
        for field in fields:
            dic[field] = analyzer.cutAndFilter(doc[field])
        # 添加新闻类别
        dic['doc_type'] = doc['doc_type']
        tokens.append(dic)
    return tokens

# 对新闻标题进行分词，得到带分词的新闻数据
tokenSougouNews = featurelize(sougouNews, fields=['doc_title'], analyzer=Analyzer())
print('完成对新闻标题的分词')

# 将分词后的结果dump到本地
with open('tokenSougouNews.pk', 'wb') as f:
    f.truncate()
    pickle.dump(tokenSougouNews, f)
print('成功将分词后的数据dump到本地')

# 划分训练集和测试集
random.shuffle(tokenSougouNews)
trainPercent = 0.8
# dump训练集
with open('tokenSougouNews-train.pk', 'wb') as f:
    f.truncate()
    pickle.dump(tokenSougouNews[:int(trainPercent*len(tokenSougouNews))], f)
print('成功dump训练集到本地：size=%d' % (int(trainPercent*len(tokenSougouNews))))
    
# dump测试集
with open('tokenSougouNews-test.pk', 'wb') as f:
    f.truncate()
    pickle.dump(tokenSougouNews[int(trainPercent*len(tokenSougouNews)):], f)
print('成功dump测试集到本地：size=%d' % (len(tokenSougouNews) - int(trainPercent*len(tokenSougouNews))))

tokenSougouNews[0]

成功导出新闻数据：size=103320
完成对新闻标题的分词
成功将分词后的数据dump到本地
成功dump训练集到本地：size=82656
成功dump测试集到本地：size=20664


{'doc_title': ['下水道', '替代', '方案', '蓄洪', '池', '密码'], 'doc_type': '财经'}

In [6]:
from sklearn.cross_validation import train_test_split
import random

X = [1,2,3,4]
random.shuffle(X)
X

[1, 3, 2, 4]

In [10]:
X[:1]

[1]